<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Получите-данные-и-загрузите-их-в-рабочую-среду.-(Jupyter-Notebook-или-другую)" data-toc-modified-id="Получите-данные-и-загрузите-их-в-рабочую-среду.-(Jupyter-Notebook-или-другую)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)</a></span></li><li><span><a href="#Подготовьте-датасет-к-обучению-моделей:" data-toc-modified-id="Подготовьте-датасет-к-обучению-моделей:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовьте датасет к обучению моделей:</a></span></li><li><span><a href="#Разделите-выборку-на-обучающее-и-тестовое-подмножество.-80%-данных-оставить-на-обучающее-множество,-20%-на-тестовое." data-toc-modified-id="Разделите-выборку-на-обучающее-и-тестовое-подмножество.-80%-данных-оставить-на-обучающее-множество,-20%-на-тестовое.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.</a></span></li><li><span><a href="#Обучите-модель-логистической-регрессии-с-параметрами-по-умолчанию." data-toc-modified-id="Обучите-модель-логистической-регрессии-с-параметрами-по-умолчанию.-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучите модель логистической регрессии с параметрами по умолчанию.</a></span></li><li><span><a href="#Подсчитайте-основные-метрики-модели." data-toc-modified-id="Подсчитайте-основные-метрики-модели.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Подсчитайте основные метрики модели.</a></span></li><li><span><a href="#Оптимизируйте-3-4-параметра-модели:" data-toc-modified-id="Оптимизируйте-3-4-параметра-модели:-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Оптимизируйте 3-4 параметра модели:</a></span></li><li><span><a href="#Сформулируйте-выводы-по-проделанной-работе:" data-toc-modified-id="Сформулируйте-выводы-по-проделанной-работе:-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Сформулируйте выводы по проделанной работе:</a></span></li></ul></div>

**Задание**

**Цель:**
Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.

**Описание задания:**

В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с [сайта](https://www.kaggle.com/fedesoriano/heart-failure-prediction). Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import uniform

import warnings
warnings.filterwarnings("ignore")

# Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [2]:
df = pd.read_csv('data/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [4]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


# Подготовьте датасет к обучению моделей:
a) Категориальные переменные переведите в цифровые значения. Можно использовать [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html), [preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). Старайтесь не использовать для этой задачи циклы.

In [5]:
X = pd.get_dummies(df, drop_first=True).drop('HeartDisease', axis=1)
y = df['HeartDisease']
X.shape, y.shape

((918, 15), (918,))

# Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
X_train.shape, X_test.shape

((734, 15), (184, 15))

# Обучите модель логистической регрессии с параметрами по умолчанию.

In [7]:
clf = LogisticRegression(random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

LogisticRegression(n_jobs=-1, random_state=42)

# Подсчитайте основные метрики модели. 
Используйте следующие метрики и функцию: cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])

In [8]:
def mean_scores(clf, X, y, index):
    '''Функция считает средние значения метрик, полученных при помощи кросс-валидации и возвращает pd.DataFrame'''
    dict_ = cross_validate(clf, X, y, cv=10, scoring=['accuracy','recall','precision','f1'], n_jobs=-1)
    res = dict()
    res['fit_time'] =np.mean(dict_['fit_time'])
    res['score_time'] =np.mean(dict_['score_time'])
    res['test_accuracy'] =np.mean(dict_['test_accuracy'])
    res['test_recall'] =np.mean(dict_['test_recall'])
    res['test_precision'] =np.mean(dict_['test_precision'])
    res['test_f1'] =np.mean(dict_['test_f1'])
    return pd.DataFrame(res, index=[index])

In [9]:
scores = mean_scores(clf, X, y, 'baseline')
scores

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
baseline,0.06706,0.008092,0.848387,0.865608,0.86598,0.861264


# Оптимизируйте 3-4 параметра модели:

**a) Используйте GridSearchCV.**

In [10]:
params_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'l1_ratio': [x/100 for x in range(1, 30)],
    'C': [x/10 for x in range(1, 30)]
}

In [11]:
%%time
clf_grid = LogisticRegression(random_state=42, n_jobs=-1)
grid = GridSearchCV(clf_grid, params_grid, scoring='f1', cv=10, n_jobs=-1)
scores = pd.concat([scores, mean_scores(grid, X, y, 'GridSearchCV')])
scores

CPU times: total: 31.2 ms
Wall time: 15min 25s


,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
baseline,0.067060,0.008092,0.848387,0.865608,0.86598,0.861264
GridSearchCV,514.209143,0.007800,0.844040,0.865608,0.85904,0.857716


**b) Используйте RandomizedSearchCV.**

In [12]:
params_rand = {
    'tol': uniform(loc=0, scale=1e-3),
    'C': uniform(loc=0, scale=4),
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}

In [13]:
%%time
clf_rand = LogisticRegression(random_state=42, n_jobs=-1)
rand = RandomizedSearchCV(clf_rand, params_rand, scoring='f1', cv=10, n_iter=50, random_state=42, n_jobs=-1)
scores = pd.concat([scores, mean_scores(rand, X, y, 'RandomizedSearchCV')])
scores

CPU times: total: 93.8 ms
Wall time: 31 s


,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
baseline,0.067060,0.008092,0.848387,0.865608,0.865980,0.861264
GridSearchCV,514.209143,0.007800,0.844040,0.865608,0.859040,0.857716
RandomizedSearchCV,17.824366,0.013040,0.851660,0.865608,0.870611,0.863669


**c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.**

**d) Повторите п. 5 после каждого итогового изменения параметров.**

In [14]:
params_rf = {
    'n_estimators': range(10, 100, 10),
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_samples': uniform()
}
params_boost = {
    'learning_rate': uniform(loc=0.01, scale=0.3),
    'n_estimators': range(50, 300, 50),
    'max_depth': range(2, 11)
}
params_knn = {
    'n_neighbors': range(2, 10),
    'leaf_size': range(2, 50, 10),
    'p': range(1, 6)
}

In [15]:
%%time
rf = RandomizedSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    params_rf,
    scoring='f1', cv=10, n_iter=50, random_state=42, n_jobs=-1
)
scores = pd.concat([scores, mean_scores(rf, X, y, 'RandomForestClassifier')])

boost = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=42),
    params_boost,
    scoring='f1', cv=10, n_iter=50, random_state=42, n_jobs=-1
)
scores = pd.concat([scores, mean_scores(boost, X, y, 'GradientBoostingClassifier')])

knn = RandomizedSearchCV(
    KNeighborsClassifier(n_jobs=-1),
    params_knn,
    scoring='f1', cv=10, n_iter=50, random_state=42, n_jobs=-1
)
scores = pd.concat([scores, mean_scores(knn, X, y, 'KNeighborsClassifier')])

CPU times: total: 141 ms
Wall time: 21min 30s


# Сформулируйте выводы по проделанной работе:
a) Сравните метрики построенных моделей.

In [16]:
scores

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
baseline,0.067060,0.008092,0.848387,0.865608,0.865980,0.861264
GridSearchCV,514.209143,0.007800,0.844040,0.865608,0.859040,0.857716
RandomizedSearchCV,17.824366,0.013040,0.851660,0.865608,0.870611,0.863669
RandomForestClassifier,53.259237,0.023377,0.844052,0.853804,0.864741,0.855521
GradientBoostingClassifier,644.174080,0.141751,0.857131,0.871490,0.875186,0.869245
KNeighborsClassifier,10.107106,0.030203,0.734018,0.776627,0.758201,0.755042


Более высокие метрики RandomizedSearchCV, по сравнению с GridSearchCV, связаны с использованием других гиперпараметров и, возможно, неоптимальной сеткой.

b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

![Метрики из домашнего задания по теме «Ансамблирование»](data/Screenshot_4.png)

В данной работе метрики у RandomForestClassifier получились несколько ниже, т.к. в домашнем задании по теме «Ансамблирование» проводилась оптимизация с помощью GridSearchCV. 